In [1]:
%pwd

'/home/robak/Dokumenty/LabelThisSongForMe/notebooks'

In [2]:
from models.common import load_file_lists
from models.preprocessor import OpenL3PreProcessor
import numpy as np
import os

In [3]:
p = OpenL3PreProcessor(input_path="../data/mtat/mp3",
                       output_path="../data/mtat/emb",
                       suffix="npy")
# print(load_file_lists(["../split/mtat/train.npy", "../split/mtat/valid.npy", "../split/mtat/test.npy"])[:, 1])
p.run(files=load_file_lists(["../split/mtat/train.npy", "../split/mtat/valid.npy", "../split/mtat/test.npy"])[:, 1])

 55%|█████▌    | 11630/21106 [00:00<00:00, 116292.07it/s]/home/robak/Dokumenty/LabelThisSongForMe/venv/lib/python3.10/site-packages/torchopenl3/models.py:200: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = librosa.filters.mel(
100%|██████████| 21106/21106 [15:40<00:00, 22.44it/s]    


In [4]:
data = load_file_lists(["../split/mtat/train.npy", "../split/mtat/valid.npy", "../split/mtat/test.npy"])
binary = np.load("../split/mtat/binary.npy", allow_pickle=True)

In [9]:
import pathlib

X = []
Y = []
for idx, filename in data:
    filename = os.path.join("../data/mtat/emb", str(pathlib.Path(filename).with_suffix(".npy")))
    file_data = np.load(filename, allow_pickle=True).flatten()
    X.append(file_data)
    Y.append(binary[int(idx)])

In [10]:
X

[array([2.4309678, 2.0897245, 3.3490872, ..., 1.1936121, 3.2180316,
        4.0102696], dtype=float32),
 array([2.4309678, 1.8223073, 4.920432 , ..., 1.3503028, 3.0894313,
        1.9410709], dtype=float32),
 array([2.4309678, 2.1078594, 3.1710026, ..., 1.2683772, 3.0523992,
        2.843211 ], dtype=float32),
 array([2.5829554, 2.05883  , 3.2187934, ..., 1.2954909, 3.292823 ,
        2.0935686], dtype=float32),
 array([2.4309678, 2.5273156, 3.4051354, ..., 2.1659622, 3.1694639,
        2.2498198], dtype=float32),
 array([2.5750644, 2.2975128, 2.6468427, ..., 2.0301933, 3.0889337,
        3.0981016], dtype=float32),
 array([2.643259 , 3.2049153, 3.1869085, ..., 1.2759452, 2.9596565,
        2.0692663], dtype=float32),
 array([2.4309678, 2.2097952, 3.4939978, ..., 0.5654964, 3.4342184,
        3.3962874], dtype=float32),
 array([2.4309678, 2.064663 , 3.126065 , ..., 1.7705113, 3.1977398,
        3.631367 ], dtype=float32),
 array([2.4309678, 2.5538929, 3.1190946, ..., 2.27015  , 3.22582

In [11]:
Y

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

In [23]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(bootstrap=True,
                             max_depth=10,
                             max_features='sqrt',
                             random_state=1)
rfc.fit(X_train, Y_train)

RandomForestClassifier(max_depth=10, random_state=1)

In [ ]:
from sklearn.metrics import classification_report

y_pred = rfc.predict(X_test)
print(classification_report(y_pred, Y_test))